In [98]:
# импортируем всё необходимое с учетом настроек с семинара
import pandas as pd
import seaborn as sn
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import os
# Вывод таблицы в удобном формате
from tabulate import tabulate
from scipy.stats import norm, t
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(suppress=True)
import matplotlib.pyplot as pt
pt.rcParams["font.size"] = 14
pt.rcParams['figure.figsize'] = (15, 8)
sn.set_style("whitegrid")

# Задача 2

## 1. Прочтение файла

In [4]:
expense = pd.read_csv(r'C:\Users\admin\metrix1-2024-hw2\expend.csv', sep = ',') # учитываем, что разделитель запятая (хотя вроде это автоматически тоже учлось бы)
expense

,f3,v3,tot3,prov,reg,scl,nahm,durb,nch06,nch711,nch1217,nch18
0,4322.27,9022.20,45768.95,5,4,3,2,7,0,0,0,0
1,6218.96,372.95,31959.59,2,3,4,2,6,0,0,0,0
2,2636.80,565.85,20518.39,3,3,2,1,7,0,0,0,0
3,9884.05,0.00,30920.13,11,5,5,3,10,0,1,1,0
4,8718.10,4892.35,66465.31,8,2,3,3,9,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
422,8627.25,247.68,30313.82,10,5,3,2,6,2,0,0,2
423,4707.97,1521.10,41544.33,10,5,2,2,9,0,0,0,2
424,10052.16,0.00,38116.50,10,5,4,2,6,3,0,0,2
425,7539.00,2939.00,40119.12,10,5,2,2,6,0,2,0,2


## 2. Суммарные статистики tot3 и f3

In [5]:
df = expense[['tot3', 'f3']]
df = df.describe().transpose() # транспонируем полученную табличку, чтобы было красивее
df = df.rename(index={'tot3': 'Полные расходы (голл. гд)', 'f3': 'Расходы на питание (голл. гд)'}) #переименовываем
df = df.round(2) # округляем до 2-ух знаков после запятой (хотя здесь бы и так все хорошо выводилось, но нам надо, чтобы еще и в эксель норм скачивалось)
df

,count,mean,std,min,25%,50%,75%,max
Полные расходы (голл. гд),427.00,35762.81,18053.31,8728.86,23189.46,31847.12,44050.52,124564.19
Расходы на питание (голл. гд),427.00,6890.88,2745.75,883.99,4946.09,6838.43,8788.36,17270.75


Далее копирую функцию из дз1

In [6]:
from openpyxl import load_workbook # импортируем дополнительные модули (точнее из них импортируем доп. настройки), 
                                    # необходимые для более удачного оформления эконометрических таблиц
from openpyxl.styles import Border, Side, Font
def save_to_excel(file, name):
    folder = r'C:\Users\admin\metrix1-2024-hw2' # задаем сразу папку с дз, куда будут сохраняться все файлы
    filepath = os.path.join(folder, name)
    if not name.endswith('.xlsx'):
        filepath += '.xlsx' # добавляем к имени файла нужное расширение
    try:
        file = file.round(2)
        file.to_excel(filepath) # сохраняем файл по указанному пути в папку
        wb = load_workbook(filepath) # загружаем сохраненную книгу эксель для дальнейшей работы с ней
        ws = wb.active
        thin = Side(border_style="thin", color="000000") # далее настраиваем прочерчивание границ в таблице
        border = Border(left=thin, right=thin, top=thin, bottom=thin)
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = border
        
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter 
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except Exception as e:
                    pass
            adjusted_width = (max_length + 2) 
            ws.column_dimensions[column_letter].width = adjusted_width

        wb.save(filepath) # сохраняем изменения по указанному пути
        print(f'Таблица сохранена под именем: "{name}"')
    except Exception as e:
        print(f'Произошла ошибка при сохранении: {e}') # выводим описание ошибки в удобном читаемом виде, если что-то пошло не так

In [7]:
save_to_excel(df, 'суммарные статистики расходов на питание и отдых')

Таблица сохранена под именем: "суммарные статистики расходов на питание и отдых"


## 3. Суммарные статистики 4 переменных

In [8]:
expense['log_tot3'] = np.log(expense['tot3']) # добавляем с помощью модуля numpy столбцы с логарифмами
expense['log_f3'] = np.log(expense['f3'])
df1 = expense[['tot3', 'f3', 'log_tot3', 'log_f3']] # создаем новый датафрейм из нужных столбцов
df1 = df1.describe().transpose()
df1

,count,mean,std,min,25%,50%,75%,max
tot3,427.00,35762.81,18053.31,8728.86,23189.46,31847.12,44050.52,124564.19
f3,427.00,6890.88,2745.75,883.99,4946.09,6838.43,8788.36,17270.75
log_tot3,427.00,10.37,0.48,9.07,10.05,10.37,10.69,11.73
log_f3,427.00,8.74,0.48,6.78,8.51,8.83,9.08,9.76


In [9]:
save_to_excel(df1, 'суммарные статистики расходов и logs')

Таблица сохранена под именем: "суммарные статистики расходов и logs"


## 4. Парная регрессия log_f3 на log_tot3

In [10]:
y = expense['log_f3'] # задаем зависимую переменную
X = sm.add_constant(expense[['log_tot3']]) # добавляем технический коэффициент (константу) к регрессору

reg_log_f3_log_tot3 = sm.OLS(y, X).fit(cov_type='HC0') # задаем гетероскедастичность для ошибок (std errors) (по умолчанию была бы гомоскедастичность)
reg_log_f3_log_tot3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 log_f3   R-squared:                       0.312
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     163.3
Date:                Sun, 24 Nov 2024   Prob (F-statistic):           7.16e-32
Time:                        15:38:18   Log-Likelihood:                -208.53
No. Observations:                 427   AIC:                             421.1
Df Residuals:                     425   BIC:                             429.2
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0466      0.449      6.781      0.000       2.166       3.927
log_tot3       0.5492      0.043     12.779      0.000       0.465       0.633
==============================================================================
Omnibus:                       95.595   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              224.895
Skew:                          -1.128   Prob(JB):                     1.46e-49
Kurtosis:                       5.748   Cond. No.                         225.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [11]:
ci = reg_log_f3_log_tot3.conf_int().loc['log_tot3'] # строим доверительный интервал
print(f'95%-ый двусторонний доверительный интервал для эластичности по доходу (`log_tot3`): ({ci[0]}, {ci[1]})')

95%-ый двусторонний доверительный интервал для эластичности по доходу (`log_tot3`): (0.4649791320601391, 0.6334466196977857)


## 5.

У нас не принимается во внимание размер семьи. Это может привести к такой проблеме, как omitted variable bias, когда важные упущенные переменные влияют на коэффициент b1, в результате чего оценка становится смещенной.

В данном случае размер семьи предположительно влияет и на расходы на питание одной семьи (в среднем при прочих равных условиях чем больше семья, тем больше должны быть расходы), и на  доходы (в ср. при проч. равн. усл. чем больше семья, тем они должны быть больше) => будет корреляция между ошибкой нашей регрессии и регрессором. Более того, эта корреляция положительная, а значит, коэффициент b1 завышен, то есть влияние эластичности по доходу на расходы на питание выше, чем в реальности (в ср. при прочих равныъ условиях). Поскольку есть зависимость и с регрессором, и с зависимой переменной, проблема omitted variable bias действительно актуальна => оценка смещенная (в положительную сторону), модель плохая.

## 6. Регрессия с 4 регрессорами

In [29]:
y = expense['log_f3'] # задаем зависимую переменную
X = sm.add_constant(expense[['log_tot3', 'nahm', 'nch06', 'nch711']]) # добавляем технический коэффициент (константу) к регрессору

reg_log_f3_4 = sm.OLS(y, X).fit(cov_type='HC0') # задаем гетероскедастичность для ошибок (std errors) (по умолчанию была бы гомоскедастичность)
reg_log_f3_4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 log_f3   R-squared:                       0.485
Model:                            OLS   Adj. R-squared:                  0.480
Method:                 Least Squares   F-statistic:                     89.66
Date:                Sun, 24 Nov 2024   Prob (F-statistic):           4.23e-55
Time:                        17:48:31   Log-Likelihood:                -146.69
No. Observations:                 427   AIC:                             303.4
Df Residuals:                     422   BIC:                             323.7
Df Model:                           4                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4954      0.407     11.034      0.000       3.697       5.294
log_tot3       0.3595      0.041      8.733      0.000       0.279       0.440
nahm           0.2056      0.023      9.027      0.000       0.161       0.250
nch06          0.1395      0.022      6.278      0.000       0.096       0.183
nch711         0.1180      0.019      6.207      0.000       0.081       0.155
==============================================================================
Omnibus:                       69.463   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.368
Skew:                          -0.847   Prob(JB):                     1.11e-34
Kurtosis:                       5.433   Cond. No.                         244.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [13]:
c = reg_log_f3_4.conf_int().loc['log_tot3'] # строим доверительный интервал
print(f'95%-ый двусторонний доверительный интервал для эластичности по доходу (`log_tot3`): ({c[0]}, {c[1]})')

95%-ый двусторонний доверительный интервал для эластичности по доходу (`log_tot3`): (0.27882383434989233, 0.4402010747648589)


In [14]:
has_nan = expense.isnull().values.any()
print("Есть ли NaN в датасете?", has_nan)

Есть ли NaN в датасете? False


## 6 и 7: интерпретация

Сравнивая две последние регрессии (с одинаковой левой частью и сделанные по одной выборке в датасете expense. Пропущенных данных, как видно выше, нет, значит, все критерии далее можем использовать), можно сделать следующие выводы: коэффициент б1 при регрессоре log_tot3 значитеьно снизилась, ошибка тоже немного уменьшилась => мы в своем предположении были правы. При добавлении регрессоров с количеством членов семьи влияние эластичности доходов на расходы на питание действительно сократилось (теперь в среднем при прочих равных условиях увеличение доходов на 1% приводит к увеличению расходов на питание на 0,36%, а не на 0,55%). Также adjusted R^2 стал значительно выше, а значит, наша вторая модель точнее и объясняет намного больше данных. Берем именно adjusted R^2 в качестве метрики для сопоставления, поскольку он с поправкой на число регрессоров. R^2 в любом случае выше при добавлении новых регрессоров. Однако adjusted R^2 обычно не используется в качестве решающего критерия для выбора модели, поэтому анализируем дальше информационные критерии. AIC значительно снизился, что говорит в пользу второй модели. BIC тоже снизился, что тоже говорит в пользу второй модели.

F-statistic при гетероскедастичности неточная. Во второй регрессии по столбцу Z видно, что все добавленные регрессоры значимы на 5% уровне значимости (отвергаем нулевую гипотезу о их незначимости). 
Omnibus проверяет нормальность остатков регрессии. Данный показатель хоть и снизился (что говорит о распределении остатков регрессии ближе к нормальному), но гипотезу о нормальном распределении остатков на 5% уровне значимости мы отклоняем в обоих случаях.
Skew - коэффициент асимметрии остатков. Он повысился во второй модели. В обоих случаях асимметрия отрицательная (хвост распределения тяготеет в левую сторону, то есть больше значения находятся ниже среднего).
Kurtosis - центрированный нормированный четвертый момент. Во второй модели немного снизился, стал ближе к 3 (значение для нормального распределения)
JB - тест на нормальность остатков регрессии. p-value очень маленькое => отвергаем гипотезу о нормальности остатков распределения на 5% уровне значимости.
Cond.No. в обоих моделях значение достаточно высокое => может сигнализировать о проблемах с мультиколлинеарностью.
Durbin-Watson используется только для временных рядов, игонирируем

Доверительный интервал для эластичности по доходу значительно сместился влево (значит, до этого оценки действительно были завышены). Теперь мы включили в регрессию состав семьи, и он стал точнее.

Итак, в целом вторая модель удачнее: точнее объясняет и покрывает бОльшее число данных 

## 8. Влияние детей в возрасте до 6 лет и в возрасте 7-11 лет

In [17]:
regressor_names = reg_log_f3_4.params.index.to_list()
coefficients = reg_log_f3_4.params
standard_errors = reg_log_f3_4.bse
t_statistics = reg_log_f3_4.tvalues
p_values = reg_log_f3_4.pvalues

print(coefficients)

const      4.50
log_tot3   0.36
nahm       0.21
nch06      0.14
nch711     0.12
dtype: float64


In [23]:
print('H0: beta1 = beta2')
print('H1: beta1 != beta2')
alpha = 0.05
hypotheses = 'nch06 = nch711'
t_test = reg_log_f3_4.t_test(hypotheses)
print(t_test.summary())
print(t_test.pvalue)
is_hypo_rejected = t_test.pvalue < alpha
print(f'Нулевая гипотеза должна быть отвергнута на 5% уровне значимости : {is_hypo_rejected}')

H0: beta1 = beta2
H1: beta1 != beta2
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0215      0.028      0.760      0.447      -0.034       0.077
0.4474069005847374
Нулевая гипотеза должна быть отвергнута на 5% уровне значимости : False


Таким образом, мы не можем отвергнуть на 5% уровне значимости нулевую гипотезу о том, что влияние детей в возрасте до 6 лет и детей в возрасте 7-11 лет одинаково.

## 9. Добавление в регрессию (log_tot3)^2

In [24]:
expense['log_tot3s'] = expense['log_tot3']**2
expense

,f3,v3,tot3,prov,reg,scl,nahm,durb,nch06,nch711,nch1217,nch18,log_tot3,log_f3,log_tot3s
0,4322.27,9022.20,45768.95,5,4,3,2,7,0,0,0,0,10.73,8.37,115.16
1,6218.96,372.95,31959.59,2,3,4,2,6,0,0,0,0,10.37,8.74,107.58
2,2636.80,565.85,20518.39,3,3,2,1,7,0,0,0,0,9.93,7.88,98.59
3,9884.05,0.00,30920.13,11,5,5,3,10,0,1,1,0,10.34,9.20,106.90
4,8718.10,4892.35,66465.31,8,2,3,3,9,0,1,1,0,11.10,9.07,123.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,8627.25,247.68,30313.82,10,5,3,2,6,2,0,0,2,10.32,9.06,106.49
423,4707.97,1521.10,41544.33,10,5,2,2,9,0,0,0,2,10.63,8.46,113.09
424,10052.16,0.00,38116.50,10,5,4,2,6,3,0,0,2,10.55,9.22,111.27
425,7539.00,2939.00,40119.12,10,5,2,2,6,0,2,0,2,10.60,8.93,112.35


In [28]:
y = expense['log_f3'] # задаем зависимую переменную
X = sm.add_constant(expense[['log_tot3', 'log_tot3s', 'nahm', 'nch06', 'nch711']]) # добавляем технический коэффициент (константу) к регрессору

reg_log_f3s = sm.OLS(y, X).fit(cov_type='HC0') # задаем гетероскедастичность для ошибок (std errors) (по умолчанию была бы гомоскедастичность)
reg_log_f3s.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 log_f3   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.490
Method:                 Least Squares   F-statistic:                     77.27
Date:                Sun, 24 Nov 2024   Prob (F-statistic):           2.29e-57
Time:                        17:48:22   Log-Likelihood:                -141.89
No. Observations:                 427   AIC:                             295.8
Df Residuals:                     421   BIC:                             320.1
Df Model:                           5                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.6135      6.057     -2.083      0.037     -24.484      -0.743
log_tot3       3.6672      1.161      3.158      0.002       1.391       5.943
log_tot3s     -0.1594      0.056     -2.871      0.004      -0.268      -0.051
nahm           0.2007      0.023      8.686      0.000       0.155       0.246
nch06          0.1273      0.022      5.674      0.000       0.083       0.171
nch711         0.1126      0.019      5.822      0.000       0.075       0.150
==============================================================================
Omnibus:                       73.538   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              176.187
Skew:                          -0.870   Prob(JB):                     5.51e-39
Kurtosis:                       5.621   Cond. No.                     3.73e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 3.73e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Судя по Z, наша новая переменная является значимой на 5% уровне значимости, и хотя у нас появляется предупреждение о мультиколлинеарности, поскольку между двумя нашими регрессорами (log_tot3 and log_tot3s) есть зависимость, однако зависимость нелинейная, поэтому данные регрессоры могут быть в модели одновременно.

## 10. Эластичность при различных уровнях дохода tot3

In [34]:
elast = reg_log_f3s.params['log_tot3'] + 2 * reg_log_f3s.params['log_tot3s'] * np.log(36000)
grad = np.array([1, 2 * np.log(36000)])
var_elast = grad.T @ reg_log_f3s.cov_params().loc[['log_tot3', 'log_tot3s'], ['log_tot3', 'log_tot3s']] @ grad
se_elast = np.sqrt(var_elast)
print(f"Эластичность: {elast}")


Эластичность: 0.32301013467507644


## 11. Доверительный интервал

In [37]:
lower = elast - 1.96 * se_elast
upper = elast + 1.96 * se_elast
print(f"Доверительный интервал: ({lower}, {upper})")

Доверительный интервал: (0.24413285481088043, 0.40188741453927246)


## 13. Регрессия с провинциями

In [59]:
cat_cols = ['prov']
data = pd.get_dummies(expense, columns=cat_cols, dtype=float) # как на семинаре
data

,f3,v3,tot3,reg,scl,nahm,durb,nch06,nch711,nch1217,nch18,log_tot3,log_f3,log_tot3s,prov_1,prov_2,prov_3,prov_4,prov_5,prov_6,prov_7,prov_8,prov_9,prov_10,prov_11,prov_12,prov_13
0,4322.27,9022.20,45768.95,4,3,2,7,0,0,0,0,10.73,8.37,115.16,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,6218.96,372.95,31959.59,3,4,2,6,0,0,0,0,10.37,8.74,107.58,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2636.80,565.85,20518.39,3,2,1,7,0,0,0,0,9.93,7.88,98.59,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,9884.05,0.00,30920.13,5,5,3,10,0,1,1,0,10.34,9.20,106.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,8718.10,4892.35,66465.31,2,3,3,9,0,1,1,0,11.10,9.07,123.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,8627.25,247.68,30313.82,5,3,2,6,2,0,0,2,10.32,9.06,106.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
423,4707.97,1521.10,41544.33,5,2,2,9,0,0,0,2,10.63,8.46,113.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
424,10052.16,0.00,38116.50,5,4,2,6,3,0,0,2,10.55,9.22,111.27,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
425,7539.00,2939.00,40119.12,5,2,2,6,0,2,0,2,10.60,8.93,112.35,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00


In [73]:
y = data['log_f3'] # задаем зависимую переменную
prov_columns = data.filter(like='prov').columns.tolist() # список бинарных переменных
#print(prov_columns)
del prov_columns[0]
#print(prov_columns)
X = sm.add_constant(data[['log_tot3', 'nahm', 'nch06', 'nch711'] + prov_columns]) # добавляем технический коэффициент (константу) к регрессору

reg_prov = sm.OLS(y, X).fit(cov_type='HC0') # задаем гетероскедастичность для ошибок (std errors) (по умолчанию была бы гомоскедастичность)
reg_prov.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 log_f3   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     24.73
Date:                Sun, 24 Nov 2024   Prob (F-statistic):           1.78e-50
Time:                        18:22:59   Log-Likelihood:                -139.59
No. Observations:                 427   AIC:                             313.2
Df Residuals:                     410   BIC:                             382.1
Df Model:                          16                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4764      0.408     10.972      0.000       3.677       5.276
log_tot3       0.3457      0.041      8.482      0.000       0.266       0.426
nahm           0.2069      0.022      9.333      0.000       0.163       0.250
nch06          0.1345      0.022      6.164      0.000       0.092       0.177
nch711         0.1125      0.019      5.906      0.000       0.075       0.150
prov_2         0.1244      0.127      0.978      0.328      -0.125       0.374
prov_3         0.1458      0.177      0.824      0.410      -0.201       0.493
prov_4         0.2132      0.108      1.977      0.048       0.002       0.425
prov_5         0.1864      0.108      1.725      0.085      -0.025       0.398
prov_6         0.0524      0.125      0.417      0.676      -0.194       0.298
prov_7         0.2145      0.107      2.012      0.044       0.006       0.423
prov_8         0.1487      0.110      1.349      0.177      -0.067       0.365
prov_9         0.3000      0.119      2.511      0.012       0.066       0.534
prov_10        0.1957      0.105      1.858      0.063      -0.011       0.402
prov_11        0.1418      0.116      1.220      0.222      -0.086       0.370
prov_12        0.1039      0.121      0.857      0.391      -0.134       0.341
prov_13        0.2827      0.138      2.055      0.040       0.013       0.552
==============================================================================
Omnibus:                       58.407   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              128.194
Skew:                          -0.729   Prob(JB):                     1.46e-28
Kurtosis:                       5.253   Cond. No.                         255.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

Все 13 провинций в регрессоры вставлять было нельзя, поскольку тогда возникла бы проблема мультиколлинеарности (один регрессор можно было бы выразить через все остальные). Поэтому n бинарных регрессоров принято включать (n-1) регрессор.

In [86]:
hypothesis = 'prov_2 = 0, prov_3 = 0, prov_4 = 0, prov_5 = 0, prov_6 = 0, prov_7 = 0, prov_8 = 0, prov_9 = 0, prov_10 = 0, prov_11 = 0, prov_12 = 0, prov_13 = 0'
wald_test_result = reg_prov.wald_test(hypothesis, scalar = True)
print(wald_test_result)
is_hypo_rejected = wald_test_result.pvalue < alpha
print(f'Нулевая гипотеза должна быть отвергнута на 5% уровне значимости : {is_hypo_rejected}')

<Wald test (chi2): statistic=14.981109873300337, p-value=0.24247112748823818, df_denom=12>
Нулевая гипотеза должна быть отвергнута на 5% уровне значимости : False


Таким образом, мы не можем отвергнуть на уровне значимости 5% нулевую гипотезу о совместной незначимоти эффекта провинции, то есть он не оказывает статистически значимого влияния на нашу зависимую переменную на данном уровне значимости

## 13. Регрессия с классами

In [87]:
cat_cols = ['scl']
data1 = pd.get_dummies(expense, columns=cat_cols, dtype=float) # как на семинаре
data1

,f3,v3,tot3,prov,reg,nahm,durb,nch06,nch711,nch1217,nch18,log_tot3,log_f3,log_tot3s,scl_1,scl_2,scl_3,scl_4,scl_5
0,4322.27,9022.20,45768.95,5,4,2,7,0,0,0,0,10.73,8.37,115.16,0.00,0.00,1.00,0.00,0.00
1,6218.96,372.95,31959.59,2,3,2,6,0,0,0,0,10.37,8.74,107.58,0.00,0.00,0.00,1.00,0.00
2,2636.80,565.85,20518.39,3,3,1,7,0,0,0,0,9.93,7.88,98.59,0.00,1.00,0.00,0.00,0.00
3,9884.05,0.00,30920.13,11,5,3,10,0,1,1,0,10.34,9.20,106.90,0.00,0.00,0.00,0.00,1.00
4,8718.10,4892.35,66465.31,8,2,3,9,0,1,1,0,11.10,9.07,123.31,0.00,0.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,8627.25,247.68,30313.82,10,5,2,6,2,0,0,2,10.32,9.06,106.49,0.00,0.00,1.00,0.00,0.00
423,4707.97,1521.10,41544.33,10,5,2,9,0,0,0,2,10.63,8.46,113.09,0.00,1.00,0.00,0.00,0.00
424,10052.16,0.00,38116.50,10,5,2,6,3,0,0,2,10.55,9.22,111.27,0.00,0.00,0.00,1.00,0.00
425,7539.00,2939.00,40119.12,10,5,2,6,0,2,0,2,10.60,8.93,112.35,0.00,1.00,0.00,0.00,0.00


In [106]:
y = data1['log_f3'] # задаем зависимую переменную
scl_columns = data1.filter(like='scl').columns.tolist() # список бинарных переменных
#print(scl_columns)
del scl_columns[4]
#print(scl_columns)
X = sm.add_constant(data1[['log_tot3', 'nahm', 'nch06', 'nch711'] + scl_columns]) # добавляем технический коэффициент (константу) к регрессору

reg_scl = sm.OLS(y, X).fit(cov_type='HC0') # задаем гетероскедастичность для ошибок (std errors) (по умолчанию была бы гомоскедастичность)
reg_scl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 log_f3   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.487
Method:                 Least Squares   F-statistic:                     48.05
Date:                Sun, 24 Nov 2024   Prob (F-statistic):           1.12e-54
Time:                        19:44:23   Log-Likelihood:                -141.84
No. Observations:                 427   AIC:                             301.7
Df Residuals:                     418   BIC:                             338.2
Df Model:                           8                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2160      0.450      9.361      0.000       3.333       5.099
log_tot3       0.3934      0.047      8.413      0.000       0.302       0.485
nahm           0.1945      0.022      8.653      0.000       0.150       0.239
nch06          0.1409      0.022      6.405      0.000       0.098       0.184
nch711         0.1206      0.020      5.961      0.000       0.081       0.160
scl_1         -0.1234      0.087     -1.421      0.155      -0.293       0.047
scl_2         -0.1316      0.079     -1.663      0.096      -0.287       0.023
scl_3          0.0031      0.074      0.042      0.967      -0.143       0.149
scl_4         -0.0261      0.066     -0.396      0.692      -0.155       0.103
==============================================================================
Omnibus:                       65.092   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              139.427
Skew:                          -0.815   Prob(JB):                     5.29e-31
Kurtosis:                       5.276   Cond. No.                         263.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [108]:
hypothesis = 'scl_1 = 0, scl_2 = 0, scl_3 = 0, scl_4 = 0'
wald_test_result = reg_scl.wald_test(hypothesis, scalar = True)
print(wald_test_result)
is_hypo_rejected = wald_test_result.pvalue < alpha
print(f'Нулевая гипотеза должна быть отвергнута на 5% уровне значимости : {is_hypo_rejected}')

<Wald test (chi2): statistic=8.537426303712415, p-value=0.07376086276873409, df_denom=4>
Нулевая гипотеза должна быть отвергнута на 5% уровне значимости : False


Таким образом, мы не можем отвергнуть на уровне значимости 5% нулевую гипотезу о совместной незначимоти эффекта класса, то есть он не оказывает статистически значимого влияния на нашу зависимую переменную на данном уровне значимости

## 14. Таблица

In [111]:
def extract_model_results_with_significance(model, variables):
    coef = model.params
    std_err = model.bse
    p_values = model.pvalues
    results = []
    for var in variables:
        if var in coef.index:
            stars = ""
            if p_values[var] < 0.01:
                stars = "***"
            elif p_values[var] < 0.05:
                stars = "**"
            elif p_values[var] < 0.1:
                stars = "*"
            results.append(f"{coef[var]:.3f}{stars} ({std_err[var]:.3f})")
        else:
            results.append("-")  # Если переменной нет в модели
    return results

variables = ['log_tot3', 'nahm', 'nch06', 'nch711', 'const']

# Результаты для каждой модели
results_model1 = extract_model_results_with_significance(reg_log_f3_log_tot3, ['log_tot3', '-', '-', '-', 'const'])
results_model2 = extract_model_results_with_significance(reg_log_f3_4, variables)
results_model3 = extract_model_results_with_significance(reg_prov, variables)
results_model4 = extract_model_results_with_significance(reg_scl, variables)

# Дополнительная информация: провинции, классы, наблюдения, R^2
additional_info = [
    ["Нет", "Нет", int(reg_log_f3_log_tot3.nobs), f"{reg_log_f3_log_tot3.rsquared:.3f}"],
    ["Нет", "Нет", int(reg_log_f3_4.nobs), f"{reg_log_f3_4.rsquared:.3f}"],
    ["Да", "Нет", int(reg_prov.nobs), f"{reg_prov.rsquared:.3f}"],
    ["Нет", "Да", int(reg_scl.nobs), f"{reg_scl.rsquared:.3f}"]
]

# Сбор данных для таблицы
table_data = {
    'Переменные': [
        'log полных расходов', 'Число взрослых', 'Число детей до 6 лет',
        'Число детей от 7 до 11 лет', 'Константа',
        'Провинции', 'Социальные классы', 'Число наблюдений', 'R^2'
    ],
    'Модель 1': results_model1 + additional_info[0],
    'Модель 2': results_model2 + additional_info[1],
    'Модель 3': results_model3 + additional_info[2],
    'Модель 4': results_model4 + additional_info[3]
}

# Создание таблицы
final_table = pd.DataFrame(table_data)

#print(tabulate(final_table, headers='keys', tablefmt='grid'))
filepath = r'C:\Users\admin\metrix1-2024-hw2\регрессии.xlsx'

# Сохранение таблицы в эксель
final_table.to_excel(filepath, index = False)
wb = load_workbook(filepath)
ws = wb.active

thin = Side(border_style="thin", color="000000") # прочерчиваем границы и задаем их ширину
border = Border(left=thin, right=thin, top=thin, bottom=thin)
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.border = border
for column in ws.columns:
    max_length = 0
    column_letter = column[0].column_letter 
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except Exception as e:
            pass
    adjusted_width = (max_length + 2) 
    ws.column_dimensions[column_letter].width = adjusted_width

wb.save(filepath)
print(f'Таблица сохранена под именем: регрессии')

Таблица сохранена под именем: регрессии
